In [1]:
import pandas as pd

In [2]:
def read_csv_files(destination, dates, result=None):
    if result is None:
        dfs = []
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            dfs.append(df)
        result = pd.concat(dfs, axis=0, ignore_index=True)
    else:
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            result = pd.concat([result, df], axis=0, ignore_index=True)
    return result

In [3]:
dates = ["14_may_06_PM", "15_may_07_AM", "15_may_10_AM", "15_may_01_PM", "15_may_04_PM"]
dates.append("15_may_07_PM")
dates.append("16_may_07_AM")

In [4]:
df1 = read_csv_files( "Jakarta", dates)

In [5]:
def transfomed_df(df=df1):
    df['airline_company'] = df['airline_company'].str.split(",").str[0]
    df['out_stop_num'] = df['out_stop_num'].str.split(' ').str[0]
    df['in_stop_num'] = df['in_stop_num'].str.split(' ').str[0]

    split_df = df.pop('price_ticket').str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
    df = df.join(split_df)
    df['ticket_price'] = df['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

    date_cols = [col for col in df.columns if col.endswith('_date')]
    year= '2023'
    for col in date_cols:
        df[col] = pd.to_datetime(df[col] + ' ' + year, format='%b %d %Y')
    
    time_cols = [col for col in df.columns if col.endswith('_time')]
    df[time_cols] = df[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

    duration_cols = [col for col in df.columns if col.endswith('_duration')]
    df[duration_cols] = df[duration_cols].applymap(lambda x: pd.to_timedelta(x.replace('h', ' hours ').replace('m', ' min')))

    return df


In [6]:
Jakarta_cleaned_data = transfomed_df()

Two dataframes, df2 and df3, are merged. df2 includes all csv files with Jakarta dictionaries. The code has been adjusted to only retrieve Tokyo dictionaries, which are available in all files after df2.

In [7]:
dates = ["14_may_06_PM", "15_may_07_AM"]
df2 = read_csv_files( "Tokyo", dates)

In [8]:
df2 = df2.loc[df2['arr_city'] != 'CGK']
df2 = df2.reset_index(drop=True)

In [9]:
dates = ["15_may_10_AM", "15_may_01_PM", "15_may_04_PM", "15_may_07_PM"]
dates.append("16_may_07_AM")

In [10]:
df3 = read_csv_files( "Tokyo", dates)
df2 = pd.concat([df2, df3], ignore_index=True)

In [11]:
Tokyo_cleaned_data=transfomed_df(df2)